In [2]:
pip install pandas torch transformers numpy scikit-learn matplotlib seaborn nltk tqdm spacy textblob datasets 

   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.5 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.5 MB ? eta -:--:--
   ------------- -------------------------- 0.5/1.5 MB 985.5 kB/s eta 0:00:01
   -------------------- ------------------- 0.8/1.5 MB 985.5 kB/s eta 0:00:01
   --------------------------- ------------ 1.0/1.5 MB 1.0 MB/s eta 0:00:01
   ---------------------------------- ----- 1.3/1.5 MB 1.1 MB/s eta 0:00:01
   ---------------------------------------- 1.5/1.5 MB 1.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/626.3 kB ? eta -:--:--
   ---------------------------------------- 0.0/626.3 kB ? eta -:--:--
   ---------------- ----------------------- 262.1/626.3 kB ? eta -:--:--
   --------------------------------- ------ 524.3/626.3 

In [ ]:
import pandas as pd
df=pd.read_csv('../data/consolidado_dataset.csv', encoding='latin1')
df.head(10)

In [ ]:
import spacy
from textblob import TextBlob
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.utils.class_weight import compute_class_weight
from torch.utils.tensorboard import SummaryWriter
from datasets import Dataset
import pandas as pd
import numpy as np

# Cargar el modelo de SpaCy en español
nlp = spacy.load("es_core_news_sm")

# Inicializar TensorBoard
writer = SummaryWriter(log_dir='..logs/runs/BERT_multi_clasificacion')

# Cargar modelo y tokenizador BERT para análisis de sentimientos
tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment", num_labels=4)

# Función para preprocesar texto con SpaCy (lemmatización y stopwords)
def preprocesar_texto(texto):
    doc = nlp(texto)
    texto_lemmatizado = " ".join([token.lemma_ for token in doc if not token.is_stop and not token.is_punct])
    return texto_lemmatizado

# Función para obtener sentimiento con BERT y TextBlob
def obtener_sentimiento(texto):
    # Preprocesar con SpaCy
    texto_preprocesado = preprocesar_texto(texto)
    
    # Análisis de sentimiento con BERT
    inputs = tokenizer(texto_preprocesado, return_tensors="pt", truncation=True, padding=True)
    outputs = model(**inputs)
    predicciones = torch.softmax(outputs.logits, dim=-1)
    sentimiento_bert = torch.argmax(predicciones).item()
    
    # Complementar con TextBlob (polaridad y subjetividad)
    blob = TextBlob(texto)
    polaridad = blob.sentiment.polarity
    subjetividad = blob.sentiment.subjectivity
    
    # Mapear el sentimiento de BERT
    sentimiento_map = {0: "Muy negativo", 1: "Negativo", 2: "Neutral", 3: "Positivo", 4: "Muy positivo"}
    
    # Resultado combinado
    resultado = {
        "sentimiento_bert": sentimiento_map[sentimiento_bert],
        "polaridad_textblob": polaridad,
        "subjetividad_textblob": subjetividad
    }
    
    return resultado

# Aplicar análisis de sentimiento al texto preprocesado
df['analisis_sentimiento'] = df['COMENTARIO'].apply(obtener_sentimiento)

# Convertir los datos a Dataset de Hugging Face
def preparar_dataset(df):
    return Dataset.from_pandas(df)

# Función para tokenizar y preparar los datos para BERT
def tokenizar_y_preparar_datos(df):
    comentarios = df['comentario_preprocesado'].tolist()
    tokenized = tokenizer(comentarios, truncation=True, padding=True, max_length=128)
    # Añadir las etiquetas al dataset tokenizado
    tokenized["labels"] = list(zip(df['sentimiento'].tolist(), df['SEGMENTO'].tolist(), df['PAIS'].tolist(), df['PERIODO'].tolist()))
    return tokenized

# Convertir el dataset a Hugging Face Dataset
dataset = preparar_dataset(df)
tokenized_dataset = dataset.map(tokenizar_y_preparar_datos, batched=True)

# Dividir los datos en train y test
train_test_split = tokenized_dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']

# Función para calcular pesos de clase
def obtener_pesos_clase(y_train):
    clases = np.unique(y_train)
    pesos = compute_class_weight('balanced', classes=clases, y=y_train)
    return dict(zip(clases, pesos))

# Definir pesos de clase para cada etiqueta
pesos_sentimiento = obtener_pesos_clase(df['sentimiento'])
pesos_segmento = obtener_pesos_clase(df['SEGMENTO'])
pesos_pais = obtener_pesos_clase(df['PAIS'])
pesos_periodo = obtener_pesos_clase(df['PERIODO'])

# Configuración del entrenamiento con BERT
training_args = TrainingArguments(
    output_dir='../results',
    evaluation_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=7,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_dir='../logs',
    logging_steps=10,
    save_steps=500,
    load_best_model_at_end=True,  # Cargar el mejor modelo después del entrenamiento
    report_to="tensorboard"  # Para monitorear en TensorBoard
)

# Entrenador usando Hugging Face Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
)

# Entrenar el modelo
trainer.train()

# Evaluar y obtener las métricas
metrics = trainer.evaluate()
print(f"Resultados de evaluación: {metrics}")

# Visualización en TensorBoard
writer.add_scalar("eval_loss", metrics['eval_loss'], 0)
writer.add_scalar("eval_accuracy", metrics['eval_accuracy'], 0)

# Cerrar el escritor de TensorBoard
writer.close()

# Guardar el modelo entrenado
model.save_pretrained("../models/modelo_BERT_multi_clasificacion")
tokenizer.save_pretrained("../models/tokenizer_BERT_multi_clasificacion")
